In [87]:
import requests
from lxml import html
from lxml import etree as et
import re

from requests.exceptions import HTTPError

In [134]:
url = 'https://ficbook.net/readfic/{}'
fic_id = 8957213  # id автора по которому можно обратиться к его профилю  8957213
referer = url.format(fic_id)

In [135]:
print(url.format(fic_id))

https://ficbook.net/readfic/8957213


In [136]:
requests.get(referer)

<Response [200]>

In [139]:
# будет проверкой на ответ сервера
# https://requests.readthedocs.io/en/master/_modules/requests/exceptions/
def status(url):
    try:
        response = requests.get(url)
        # если ответ успешен, исключения задействованы не будут
        response.raise_for_status()
        
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}') # 404
        
    except ConnectionError as сonn_err:
        print(f'A Connection error occurred: {сonn_err}') 
        
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        return 1

In [140]:
status(referer)

1

In [7]:
# получить текстовый вывод 
#response = requests.get(url)
#response.encoding = 'utf-8'
#a = response.text
#a = a.split('\n')

In [141]:
response = requests.get(referer)
# Преобразование тела документа в дерево элементов (DOM)
parsed_body = html.fromstring(response.text)

##  получить title страницы

In [176]:
# Выполнение xpath в дереве элементов
title_name = str(parsed_body.xpath('//title/text()')[0])  # Получить title страницы
#print(parsed_body.xpath('//a/@href'))         # Получить аттрибут href для всех ссылок
title_name = re.sub("\s\s+", "", title_name) # https://www.rupython.com/1283-1283.html удаление нескольких пробелов
title_name

'Его искали, а он нашелся—ориджинал'

In [177]:
# таким образом можно вытащить размер

In [178]:
class_name = 'mb-5'
els = parsed_body.find_class(class_name)
els

[<Element div at 0x7f705d0c1950>,
 <Element div at 0x7f705d0bfa10>,
 <Element div at 0x7f7056103d70>,
 <Element div at 0x7f7056103d10>,
 <Element div at 0x7f7056103dd0>]

In [211]:
size = str(els[0].text_content()) 
size = re.sub("\s\s+", " ", size)
print(size)
pages, parts = re.findall("\d+", size)
print('страниц:', pages)
print('частей:', parts)

 Размер: планируется Миди, написано 365 страниц, 16 частей 
страниц: 365
частей: 16


In [209]:
tags = str(els[1].text_content()) # Метки
tags = re.sub("\s\s+", " ", tags)
tags

' Метки: PWP Попаданчество Фэнтези Экшн Элементы гета Элементы фемслэша '

In [210]:
description = str(els[2].text_content())  # Описание
description = re.sub("\s\s+", " ", description)
description

' Описание: Одной из аксиом любой истории о Героях и Злодеях, будет тот факт, что Злодей всегда будет искать Героя, дабы злодейски того уничтожить. Закономерно, эти персонажи забывают о том, что согласно итогу этих историй, искать Героя не стоит, даже если Злодей искренне считает самого себя Героем. Ведь Герой может и найтись. '

In [212]:
notes = str(els[3].text_content()) #Примечания автора
notes = re.sub("\s\s+", " ", notes)
notes

' Примечания автора: Часть третья. Правила и предупреждения аналогичны первой и второй частям. Для тех, кто не знает о тех опусах, но был бы не против их прочесть, вам в этом направлении: 1. https://ficbook.net/readfic/8205186 2. https://ficbook.net/readfic/8534650 Главы будут выходить реже, причем намного - работы завались, а времени в сутках столько же. Идеал - одна глава в неделю. Стандарт - глава в две недели. Понеслись. Добрый человек под ником ZoomGuy сделал документ с ЧаВО, за что ему огромное спасибо. Собственно, документ: https://drive.google.com/file/d/1It0OeEugeGuR8hKJ-y-PIlhlvhzMcMlK/view Р.С. Неожиданно. №1 в топе «Джен» №2 в топе «Все» '

In [213]:
publication = str(els[4].text_content()) # Публикация на других ресурсах
publication = re.sub("\s\s+", " ", publication)
publication

' Публикация на других ресурсах: Уточнять у автора/переводчика '

## Проверка коментариев 

In [215]:
last_comment = 'https://ficbook.net/readfic/{}/comments#content'
referer_last_comment = last_comment.format(fic_id)
referer_last_comment

'https://ficbook.net/readfic/8957213/comments#content'

In [216]:
response_last_comment = requests.get(referer_last_comment)
# Преобразование тела документа в дерево элементов (DOM)
parsed_last_comment = html.fromstring(response_last_comment.text)

In [217]:
parsed_last_comment

<Element html at 0x7f70560da7d0>

In [222]:
class_comment_message = 'comment_message urlize'
els_message = parsed_last_comment.find_class(class_comment_message)
for comment in els_message:
    publication = str(comment.text_content()) 
    #publication = re.sub("\s\s+", " ", publication)
    print('     ')
    print(publication)

     
Здравствуйте, коллеги-читатели!
Боюсь, за обсуждением эпичной битвы Лосия мы пропустили, стратегически, очень важную информацию. Тин принял принципиальное решение обзавестись бэнефикой. Думаю, все помнят, что случилось, когда Тин размышлял, что ему, вроде как полагается эльфийка. Сейчас же он твердо решил. Вам еще не страшно за империю Веков?
Я вижу ситуацию так:
- Тин подменяет «почтенного хозяина» его отражением. Уходя, герои решают обнести его сокровищницу (ну, чтобы не возвращаться), и находят там статую древней волшебницы/темной целительницы/еще какой стремный класс, аля Эленай Зиновия. Её либо расколдуют, либо её душа получит новое тело (а вот не надо было Призванного водить. Тина это не оскорбило. ВОТ СОВСЕМ!!!);
- при разграблении Библиотеки Тин находит саркофаг (не разрушимую и вечную темницу) очень древней императрицы и верховной жрицы ныне мёртвого (а слуга живого бога ГГ не подходит), но даже по Алурейским меркам отбитого на все голову бога. Вполне возможно, что когда

## Проверка может ли ответить сервер JSON

как заявил Мартийн Питерс в комментариях, серверы могут отвечать JSON в случае ошибок (это зависит от реализации), поэтому проверка заголовка Content-Type более надежна.
https://coderoad.ru/16573332/JSONDecodeError-%D0%BE%D0%B6%D0%B8%D0%B4%D0%B0%D0%B5%D0%BC%D0%BE%D0%B5-%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%81%D1%82%D1%80%D0%BE%D0%BA%D0%B0-1-%D1%81%D1%82%D0%BE%D0%BB%D0%B1%D0%B5%D1%86-1-%D1%81%D0%B8%D0%BC%D0%B2%D0%BE%D0%BB-0
https://github.com/psf/requests/issues/4908

In [54]:
import requests


url = 'https://ficbook.net/readfic/8957213'
r = requests.get(url)

if 'json' in r.headers.get('Content-Type'):
    js = r.json()
else:
    print('Response content is not in JSON format.')
    js = 'spam'


Response content is not in JSON format.


In [55]:
r.headers.get('Content-Type')

'text/html; charset=UTF-8'

In [ ]:
# тест headers Content-Type, если есть в Content-Type json значит можно вывести в json

In [ ]:
#https://developer.mozilla.org/ru/docs/Web/HTTP/%D0%97%D0%B0%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BA%D0%B8/Content-Type
#https://developer.mozilla.org/ru/docs/%D0%A1%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C/MIME_type

In [63]:
test_url = 'https://api.github.com'
test = requests.get(test_url)
test.headers.get('Content-Type')
test.json()

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '